# Installations and Imports


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install transformers
!pip install accelerate
!pip install sentencepiece
!pip install datasets

In [3]:
from transformers import AutoModel
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc
import sentencepiece
from torch.utils.data import Dataset, DataLoader
import re
from tqdm import tqdm
from accelerate import Accelerator
import pickle

import os

In [4]:
SEED = 1111
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#Dataset preparation

In [5]:
import json
with open(f"/content/drive/MyDrive/shroom/val.model-agnostic.json", "r") as outfile:
  val_model_agnostic=json.load(outfile)
print(len(val_model_agnostic))

val_model_agnostic_df=pd.DataFrame(data=val_model_agnostic, columns=["src", "tgt", "hyp", "ref", "task", "model", "labels", "label", "p(Hallucination)"])
val_model_agnostic_df.head(4)

499


,src,tgt,hyp,ref,task,model,labels,label,p(Hallucination)
0,The writer had just entered into his eighteent...,Resembling a weasel (in appearance).,Resembling or characteristic of a weasel.,tgt,DM,,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.2
1,Sailors ' and fishermen 's <define> sheath - k...,.,Alternative form of sheath knife,tgt,DM,,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,0.8
2,"As to age , Bead could not form any clear impr...","(poetic) An instant, a short moment.",(obsolete) A short period of time.,tgt,DM,,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
3,Because redpillers are usually normies or <def...,"(incel, _, slang) A man of a slightly lower ra...",(slang) An incel.,tgt,DM,,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.2


In [6]:
gold_labels=[]
first_sentences=[]
second_sentences=[]

for i in tqdm(range(len(val_model_agnostic_df))):
  sample=val_model_agnostic_df.iloc[i]
  src=sample['src']
  tgt=sample['tgt']
  ref=sample['ref']
  hyp=sample['hyp']
  task=sample['task']
  label=sample['label'].strip().lower()

  if label=="hallucination":
    label="contradiction"
  else:
    label="entailment"

  if task=="MT" or task=="DM":
    sentence1=sample['tgt'].strip().lower()
    sentence2=sample['hyp'].strip().lower()
  else:
    sentence1=sample['src'].strip().lower()
    sentence2=sample['tgt'].strip().lower()

  assert len(sentence1)>0
  assert len(sentence2)>0

  gold_labels.append(label)
  first_sentences.append(sentence1)
  second_sentences.append(sentence2)


100%|██████████| 499/499 [00:00<00:00, 4476.43it/s]


In [7]:
with open(f"/content/drive/MyDrive/shroom/val.model-aware.v2.json", "r") as outfile:
  val_model_aware=json.load(outfile)
print(len(val_model_aware))

val_model_aware_df=pd.DataFrame(data=val_model_aware, columns=["src", "tgt", "hyp", "ref", "task", "model", "labels", "label", "p(Hallucination)"])
val_model_aware_df.head(4)

501


,src,tgt,hyp,ref,task,model,labels,label,p(Hallucination)
0,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,A sloping top .,tgt,DM,ltg/flan-t5-definition-en-base,"[Not Hallucination, Hallucination, Not Halluci...",Hallucination,0.6
1,Please try not to overreact if she drives badl...,To react too much or too intensely .,To react too much .,tgt,DM,ltg/flan-t5-definition-en-base,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
2,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,The process of spoiling ; the state of being s...,tgt,DM,ltg/flan-t5-definition-en-base,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
3,The way the opposition has framed the argument...,To construct in words so as to establish a con...,To arrange in a particular way .,tgt,DM,ltg/flan-t5-definition-en-base,"[Hallucination, Not Hallucination, Not Halluci...",Hallucination,0.6


In [8]:
for i in tqdm(range(len(val_model_aware_df))):
  sample=val_model_aware_df.iloc[i]
  src=sample['src']
  tgt=sample['tgt']
  ref=sample['ref']
  hyp=sample['hyp']
  task=sample['task']
  label=sample['label'].strip().lower()

  if label=="hallucination":
    label="contradiction"
  else:
    label="entailment"

  if task=="MT" or task=="DM":
    sentence1=sample['tgt'].strip().lower()
    sentence2=sample['hyp'].strip().lower()
  else:
    sentence1=sample['src'].strip().lower()
    sentence2=sample['hyp'].strip().lower()

  assert len(sentence1)>0
  assert len(sentence2)>0

  gold_labels.append(label)
  first_sentences.append(sentence1)
  second_sentences.append(sentence2)


100%|██████████| 501/501 [00:00<00:00, 8632.28it/s]


In [9]:
temp_dict={
    'gold_label':gold_labels,
    'sentence1':first_sentences,
    'sentence2':second_sentences
}

train_df_part1=pd.DataFrame(data=temp_dict)
train_df_part1.head(3)

,gold_label,sentence1,sentence2
0,entailment,resembling a weasel (in appearance).,resembling or characteristic of a weasel.
1,contradiction,.,alternative form of sheath knife
2,entailment,"(poetic) an instant, a short moment.",(obsolete) a short period of time.


In [10]:
new_train_df_part1=train_df_part1.sample(frac=0.8, random_state=60)
df_dev=train_df_part1.sample(frac=0.2, random_state=60)
print(len(new_train_df_part1), len(df_dev))

800 200


In [11]:
max_sentence_len=100
def trim_sentence(sent):
    try:
        sent = sent.split()
        sent = sent[:max_sentence_len]
        return " ".join(sent)
    except:
        return sent

In [12]:
new_train_df_part1['sentence1'] = new_train_df_part1['sentence1'].apply(trim_sentence)
new_train_df_part1['sentence2'] = new_train_df_part1['sentence2'].apply(trim_sentence)
df_dev['sentence1'] = df_dev['sentence1'].apply(trim_sentence)
df_dev['sentence2'] = df_dev['sentence2'].apply(trim_sentence)

new_train_df_part1 = new_train_df_part1.loc[new_train_df_part1['gold_label'].isin(['entailment','contradiction'])]
df_dev = df_dev.loc[df_dev['gold_label'].isin(['entailment','contradiction'])]

print(len(new_train_df_part1), len(df_dev))

800 200


add new labeled data

In [13]:
path='/content/drive/MyDrive/shroom/our_final_shroom_models/train.model-agnostic_labeled.csv'

train_model_agnostic_labeled=pd.read_csv(path)
train_model_agnostic_labeled.head()

,src,tgt,hyp,ref,task,model,label
0,Не волнуйся. Это только временно.,Don't worry. It's only temporary.,"Don't worry, it's only temporary.",either,MT,NaN,0
1,"Тома никогда нет там, где он должен быть.",Tom is never where he's supposed to be.,Tom is never where he should be.,either,MT,NaN,0
2,Мне сложно работать с Томом.,I have trouble working with Tom.,It's hard for me to work with Tom.,either,MT,NaN,0
3,"Воду, пожалуйста.",I'd like some water.,"Water, please.",either,MT,NaN,0
4,"Я не ожидал, что Том предаст меня.",I didn't think that Tom would betray me.,I didn't expect Tom to betray me.,either,MT,NaN,0


In [14]:
gold_labels=[]
first_sentences=[]
second_sentences=[]

for i in tqdm(range(len(train_model_agnostic_labeled))):
  sample=train_model_agnostic_labeled.iloc[i]
  src=sample['src']
  tgt=sample['tgt']
  ref=sample['ref']
  hyp=sample['hyp']
  task=sample['task']
  label=sample['label']

  if label==1:
    label="contradiction"
  else:
    label="entailment"

  sentence1=sample['tgt']

  if type(sentence1)!=str:
    sentence1=str(sentence1)

  sentence1=sentence1.strip().lower()
  assert len(sentence1)>0

  sentence2=sample["hyp"]

  if type(sentence2)!=str:
    sentence2=str(sentence2)

  sentence2=sentence2.strip().lower()

  assert len(sentence2)>0

  gold_labels.append(label)
  first_sentences.append(sentence1)
  second_sentences.append(sentence2)


100%|██████████| 1000/1000 [00:00<00:00, 5559.16it/s]


In [15]:
temp_dict={
    'gold_label':gold_labels,
    'sentence1':first_sentences,
    'sentence2':second_sentences
}

df=pd.DataFrame(data=temp_dict)
df

,gold_label,sentence1,sentence2
0,entailment,don't worry. it's only temporary.,"don't worry, it's only temporary."
1,entailment,tom is never where he's supposed to be.,tom is never where he should be.
2,entailment,i have trouble working with tom.,it's hard for me to work with tom.
3,entailment,i'd like some water.,"water, please."
4,entailment,i didn't think that tom would betray me.,i didn't expect tom to betray me.
...,...,...,...
995,entailment,"(figuratively, derogatory) someone who is over...",(informal) a person who is passive or submissive.
996,contradiction,monoglot,"(chiefly, australia, new zealand, colloquial) ..."
997,contradiction,alternative form of ramshackle,ransacked.
998,contradiction,"(of a person, animal, body or object) infected...",resembling or characteristic of plague.


In [16]:
df['sentence1'] = df['sentence1'].apply(trim_sentence)
df['sentence2'] = df['sentence2'].apply(trim_sentence)

df = df.loc[df['gold_label'].isin(['entailment','contradiction'])]

print(len(df))

1000


In [17]:
new_df_train=pd.concat([df, new_train_df_part1])
new_df_train

,gold_label,sentence1,sentence2
0,entailment,don't worry. it's only temporary.,"don't worry, it's only temporary."
1,entailment,tom is never where he's supposed to be.,tom is never where he should be.
2,entailment,i have trouble working with tom.,it's hard for me to work with tom.
3,entailment,i'd like some water.,"water, please."
4,entailment,i didn't think that tom would betray me.,i didn't expect tom to betray me.
...,...,...,...
430,contradiction,i think he's faking.,i think he's making a look.
762,entailment,the european union is first and foremost an ec...,moral values are the basis of the european uni...
208,contradiction,how long you've been back?,when did you get back?
590,entailment,taking place between religious organisations t...,between congregations .


In [18]:
new_df_train=new_df_train.sample(frac=1, random_state=42)
len(new_df_train)

1800

In [19]:
new_df_train.head(2)

,gold_label,sentence1,sentence2
217,contradiction,there's got to be a different way.,there has to be another way.
943,contradiction,(legal) of or pertaining to redhibition (the a...,of or pertaining to reduction.


In [20]:
labels_dict={
    "entailment":0,
    'contradiction':1,
}

In [21]:
def prepare_df(input_df):
  gold_labels=[]
  first_sents=[]
  second_sents=[]

  for i in tqdm(range(len(input_df))):
    sample=input_df.iloc[i]

    sentence1=sample['sentence1']
    sentence2=sample['sentence2']

    if type(sentence1)!=str:
      sentence1=str(sentence1)

    if type(sentence2)!=str:
      sentence2=str(sentence2)

    first_sents.append(sentence1)
    second_sents.append(sentence2)
    gold_labels.append(labels_dict[sample['gold_label']])

  temp_dict={
    'gold_label':gold_labels,
    'sentence1': first_sents,
    'sentence2': second_sents
  }
  df=pd.DataFrame(data=temp_dict)

  return df

In [22]:
train_df=prepare_df(new_df_train)
dev_df=prepare_df(df_dev)

100%|██████████| 200/200 [00:00<00:00, 18007.49it/s]


In [23]:
train_df

,gold_label,sentence1,sentence2
0,1,there's got to be a different way.,there has to be another way.
1,1,(legal) of or pertaining to redhibition (the a...,of or pertaining to reduction.
2,1,small-scale preparation of plasmid dna,alternative form of minipreparation
3,0,that's not the only thing that tom wanted.,that's not the only thing tom wanted.
4,1,(maths),alternative form of pseudodimension
...,...,...,...
1795,1,(obsolete) obscene language or writing.,"(archaic, or, dialectal, or, obsolete) bellige..."
1796,0,suspected cases of h5n1 in croatia and denmark...,possible h5n1 cases in croatia and denmark are...
1797,1,a band of contrasting color or of feathers wit...,alternative form of wing bar
1798,0,"made up of three related elements , often matc...",having three distinct characteristics .


In [24]:
from transformers import AutoTokenizer, DebertaForSequenceClassification

model_checkpoint=f"/content/drive/MyDrive/shroom/our_final_shroom_models/deberta-v3-large-nli.pt"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [25]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict({'train': Dataset.from_pandas(train_df), 'dev':Dataset.from_pandas(dev_df)})
dataset

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 1800
    })
    dev: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 200
    })
})

In [26]:
max_length = 256

def tokenize_preprocess_function(examples):

    input=tokenizer(examples['sentence1'], examples['sentence2'], max_length=max_length, truncation=True, padding=True, return_tensors="pt")

    data_dict={
        'input_ids': input['input_ids'],
        'label':torch.tensor(examples['gold_label']),
        }

    return data_dict

tokenized_datasets = dataset.map(tokenize_preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2', 'input_ids', 'label'],
        num_rows: 1800
    })
    dev: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2', 'input_ids', 'label'],
        num_rows: 200
    })
})

#Huggingface Model and Trainer

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels=2

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
from datasets import load_metric

task="mnli"
metric = load_metric('glue', task)

<ipython-input-26-2fe436dc61b3>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', task)


In [ ]:
batch_size=8
metric_name = "accuracy"
model_checkpoint = "microsoft/deberta-v3-large"
model_name = model_checkpoint.split("/")[-1]
num_epochs=10

args = TrainingArguments(
    output_dir=f"/content/{model_name}.checkpoint",
    seed=0,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_steps=200,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=True
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/shroom/{model_name}_DM_best.pt")